### Workflow: Extracting information by subtracting.

1. First, we need to decrease the "noise" as much as possible.
    - Cut content into sentences.
    - Mark sentences with kinship dictionary.
    - drop sentences without kinship words.
2. Sentence compression.
    - Some sentences have office title or place name before person name.
    - Extract office titles (marked as `"/no_noc/"`) and places (marked as `"/ns/"`) to seperate columns.
    - Compress/subtract the sentences to only include kinship titles and person names.
3. Kinship information extraction.
    - Run regular expressions on compressed sentences.
    
#### Problems and TODOs:
- [X] - "Include code IDs": 
    - Not for the relationship extraction step, since there may be duplicates of office titles/place names, complicate current tasks.
    - Therefore, no need to put the IDs in dictionaries.
- [X] - Words to be tagged:
    - Office title and place names used in Song dynasty.
    - Office titles and kinship names are overlapped, e.g., 子, 伯: Should be solved by using refined titles.
- [X] - Order effect:
    - e.g., 昭武軍節度，if meet 節度 first, will be 昭武軍/no/ after subtraction.
    - Solution: order the dictionary by length of characters, therefore, 昭武軍節度 will be matched first.
- [X] - Tagging more words:
    - [x] Update dictionaries (`no_noc`, `nk`, `vno`).
    - [x] Tag exact positions instead of combining, e.g.:
        - sent_comp: 解州解/no_noc//wsep/累贈/no_noc/諱禹偁/wsep/追封彭城/no_noc/徐氏/wsep/廣陵/no_noc/王氏/wsep/睢陽/no_noc/宋氏/wsep/祖考妣也
        - no_noc: 禮部尚書#郡太君#郡太君#郡太君#縣令
    - [x] Appointment verbs (`APPOINTMENT_TYPE_CODES`).
    - Entry data (`ENTRY_CODES`).


In [1]:
import pandas as pd
import re
from tqdm import tqdm
import math

### Prepare dictionaries.

In [2]:
song_kin_title=pd.read_excel('../../../NLP/cn_corpus/data_raw/song_kinship_title.xlsx')
song_kin_title.rename(columns={'名称':'name'}, inplace=True)
song_kin_title['pos']='nk'
song_kin_title.sample(3)

,name,关系说明,亲属编码,pos
18,五世祖,G-5,397,nk
29,曾王父,FFF,48,nk
155,侄,BS,177,nk


In [24]:
song_appt_type=pd.read_excel('../../../NLP/cn_corpus/data_raw/song_appt_type.xlsx')
song_appt_type.rename(columns={'c_appt_type_desc_chn':'name'}, inplace=True)
song_appt_type['pos']='vno'
song_appt_type.sample(3)

,c_appt_type_code,name,c_appt_type_desc,check,Unnamed: 4,Unnamed: 5,Unnamed: 6,pos
42,49,特進,NaN,NaN,NaN,NaN,NaN,vno
94,104,舉,新增,NaN,NaN,NaN,舉進士,vno
29,36,兼,NaN,NaN,NaN,NaN,NaN,vno


In [114]:
song_appt_type[['name', 'pos']].to_csv('../../../NLP/cn_corpus/data_build/cn_traditional_jieba/song_vno_dict.csv', 
                                       index=False, sep='\t')
song_kin_title[['name', 'pos']].to_csv('../../../NLP/cn_corpus/data_build/cn_traditional_jieba/song_nk_dict.csv', 
                                       index=False, sep='\t')

### Sentence compression.

#### Select sentences with kinship information.

In [2]:
# Read kinship dictionary. nk = POS Kinship.
df_nk=pd.read_csv('../data_dict/song_nk_dict.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})
# Read office title. no = office title. noc = office category.
df_no_noc=pd.read_csv('../data_dict/song_no_noc_dict.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})
df_ns=pd.read_csv('../data_dict/song_ns_dict.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})
df_vno=pd.read_csv('../data_dict/song_vno_dict.csv', sep='\t')
df_nz=pd.read_csv('../data_dict/song_nz_dict.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})
df_kin_mismatch=pd.read_csv('../data_dict/kin_mismatch.csv', sep='\t', header=None).rename(columns={0:'name'})

In [3]:
df_nk.sample(3)

,name,pos
156,第三任妻父,nk
688,仲子,nk
757,許嫁,nk


In [4]:
df_kin_mismatch.sample(3)

,name
6,子孫
10,士子
75,考等


In [5]:
df_nz.sample(3)

,name,pos
89,景文公,nz
253,獻惠公,nz
299,懿德公,nz


In [6]:
# All QSW records.
df_qsw_raw=pd.read_excel('../data_raw/quan_song_wen_muzhi.xlsx', sheet_name='墓誌銘墓表壙誌行狀神道碑塔銘墓碑')[['content_id', 'content', 'subject', 'author']].set_index('content_id')
df_qsw_raw['content']=[str(s) for s in df_qsw_raw['content']]
df_qsw_raw.drop(math.nan, inplace=True) # Drop index==NaN
# Sample a subset about 10%. Annotated after testing.
#df_qsw_raw=df_qsw_raw.sample(500)

In [7]:
# Function for seperating sentences.
def sep_mark_sent(string):
    string=string.replace('，', '/wsep/').replace('；', '/wsep/').replace('、', '/wsep/')
    string=string.replace('！', '/wend/').replace('。', '/wend/').replace('？', '/wend/').replace('！', '/wend/')
    string=string.replace('：', '/wm/')
    return [s for s in string.split("/wend/") if s!='']

In [8]:
# Seperate sentences and retain sentences with kinship words.
all_sent_count=0
kin_sent_count=0
df_qsw_refined=pd.DataFrame()
for index in tqdm(df_qsw_raw.index):
    sent_list=sep_mark_sent(df_qsw_raw.loc[index]['content']) # Seperate sentences.
    all_sent_count+=len(sent_list)
    subject=df_qsw_raw.loc[index]['subject']
    author=df_qsw_raw.loc[index]['author']
    kin_sent_list=[]
    content_id=index
    # Retain sentences with kinship information.
    for sent in sent_list:
        # Remove all kin_mismatch words first.
        sent_temp=sent
        for kin_mismatch in df_kin_mismatch['name']:
            sent_temp=sent_temp.replace(kin_mismatch, '')
        # See if kin_nm still in sentence.
        kin_nm_list=[]
        for kin_nm in df_nk['name']:
            if kin_nm in sent_temp:
                kin_nm_list.append(kin_nm)
        if kin_nm_list!=[]:
            kin_sent_list.append(sent)
    kin_sent_count+=len(kin_sent_list)
    df_qsw_refined=pd.concat([pd.DataFrame(data=[[content_id, subject, author, sent] for sent in kin_sent_list],
                                          columns=['content_id', 'subject', 'author', 'sent']
                                          ), df_qsw_refined], axis=0, ignore_index=True
                            )
print('Kinship sentences / All sentences: ', round(float(kin_sent_count)/all_sent_count, 3))
# Run on entire corpus, 21.2%.

100%|██████████| 4763/4763 [01:48<00:00, 43.76it/s]

Kinship sentences / All sentences:  0.273


In [9]:
[s for s in df_qsw_raw['content'] if '祖諱元晏' in s]

['君諱某，字某。其先始平人，在僖宗朝，有官於蜀者。廣明之亂，唐統紊裂，視世濁溺，留避於此。子孫蕃衍，有居於普者。五世祖紹卿，於五代時，以宗族門地雄於一州；高祖諱光偉，佐東川節度；曾祖諱嶠，祖諱元晏，并潛隱不仕。父諱某，少舉進士，以苦學被病，遂不顧舊業，專治養生之術。作詩百章道其事，自號丹珠子。年過八十，無疾而終。子三人，君其長也。君生而穎慧，不憙他技。未冠，求師於成都。是時，任玠溫如、李畋渭卿，皆以道義文章教授諸生，君執業門下，并爲其高第。歸，將試藝於其郡廷，以干薦書，而豪士惡子競以財賂占壓，寒素不得一步進於其下。乃退而嘆曰：‘是等也，我安能與之以力相較耶?’於是收歛退縮，芟去仕意，僻居靜處，討究群策。經深史隱，鉤擿藏詣；馳詞吐論，坐者常屈。閭里訟訴，槩先詣君所平決，以至不復更由官治，而兩講解矣。教諸子事業，悉有端次。慶曆中，其子、今中都外郎如晦，用其法一舉中進士。君曰：‘是吾門戶之大望，自此子爾。’嘉祐初，以子官授大理評事致仕，三遷爲秘書丞，賜緋衣銀魚。嘗即其居，盛創亭宇，榜之曰‘榮恩’；自作記，道其所以獲當世爲人之甚幸者。鄉人景慕之。治平二年春，中都爲晉原宰，君以雙輿就其養。晉原之治，高出一道，君實有所誨助。間則吟詩飲酒，日日不倦。一旦，召中都語之，曰：‘官居之樂，誠樂矣!然而吾之舊廬，近常往來於吾懷也。汝當具吾歸裝，宜無吾留。’十月，促就道，中都遂假檄侍還其家。既至，亟遣去，曰：‘汝速往，無以吾累汝；汝當憂民，慎毋吾憂也。’自是，日召鄉里故舊聚飲，歡嘑歗歌，愈益精健。諸子立左右，忽顧之，曰：‘父母之年，古人謂可以喜懼者，汝等當知之。吾受祿養幾二紀，名復掛朝籍，人能如吾者幾何?此可喜也。然吾春秋已高，汝能無所懼乎?’家人聞之，錯遌皇惑，問：‘何以及此?體中有覺不如平時者何所?’但俛首嘻笑，不答。又數日，食飲漸不進，求就枕，瞑目良久。以纊候其氣，已不屬矣，遂終焉，十一月十二日也。享年七十五。夫人趙氏，同郡之甲族，婉懿有善譽，宗黨模其閨法，四封爲壽光縣君。生男六人，三早夭；次，中都也；次，處晦、用晦，并舉進士，有文行聞其朋流。女五人：適昌元解惟正，都官員外郎景思問，郡人周著，進士景思永。歸思問者，先卒；後繼之以其娣，封永壽縣君。孫男十人，某某，皆嚮習文藝。孫女九人。其一始嫁河南趙仲遘〔一〕。其孤將以三年二月某日，葬君於樂至縣普安鄉之西山，從先塋也。中都與同有場屋之

In [13]:
df_qsw_refined.sample(5)

,content_id,subject,author,sent
17561,8712137.0,晏殊女、富弼妻,李藻,”嗚呼/wsep/夫人可謂終於執禮者歟
70609,8317681.0,錢俶,慎知禮,元子奉于贄生/wsep/大夫旅于庭實
36648,5654475.0,胡泳妻、李孟堅女,楊萬里,夫人慧淑莊重/wsep/容止有度/wsep/喜愠未嘗見聲氣/wsep/莊簡愛之
62107,5131878.0,郭源明,蘇頌,先是/wsep/先公侍讀既葬於縣之肥城里/wsep/餘三十年而太夫人尚無恙
11551,5307069.0,NaN,陳師道,其後仲父繫奉天獄/wsep/先君收其孥/wsep/而夫人與之有恩


In [ ]:
drop_index_list=[]
for index in tqdm(df_qsw_refined.index):
    sent=df_qsw_refined.loc[index, 'sent']
    if ('娶' in sent or '取' in sent or '配' in sent) and ('夫人' not in sent and '氏' not in sent):
        drop_index_list.append(index)
for index in tqdm(df_qsw_refined.index):
    sent=df_qsw_refined.loc[index, 'sent']
    if ('歸' in sent or '嫁' in sent or '適' in sent or '許' in sent) and ('女' not in sent and '妹' not in sent and '姑' not in sent and '夫人' not in sent and '姊' not in sent):
        drop_index_list.append(index)
        
df_qsw_refined=df_qsw_refined.drop(drop_index_list)
len(drop_index_list)

100%|██████████| 70915/70915 [00:01<00:00, 42283.06it/s]


13495

#### Subtracting information by order.

In [ ]:
# First, subtract 固定称呼 (POS: nz).
for index in tqdm(df_qsw_refined.index):
    sent=df_qsw_refined.loc[index]['sent']
    nz_list=[]
    for nz in df_nz['name']:
        if nz in sent:
            nz_list+=re.findall(nz, sent)
            sent=sent.replace(nz, '/nz/')
    df_qsw_refined.loc[index, 'sent_comp']=sent
    df_qsw_refined.loc[index, 'nz']='#'.join(nz_list)

 84%|████████▎ | 48280/57733 [08:53<01:44, 90.44it/s]

In [ ]:
# Second, subtract place names (POS:ns).
for index in tqdm(df_qsw_refined.index):
    sent=df_qsw_refined.loc[index]['sent_comp']
    ns_list=[]
    for ns in df_ns['name']:
        if ns in sent:
            ns_list+=re.findall(ns, sent)
            sent=sent.replace(ns, '/ns/')
    df_qsw_refined.loc[index, 'sent_comp']=sent
    df_qsw_refined.loc[index, 'ns']='#'.join(ns_list)

In [ ]:
# Third, subtract office title (POS:no_noc).
for index in tqdm(df_qsw_refined.index):
    sent=df_qsw_refined.loc[index]['sent_comp']
    no_noc_list=[]
    for no_noc in df_no_noc['name']:
        if no_noc in sent:
            no_noc_list+=re.findall(no_noc, sent)
            sent=sent.replace(no_noc, '/no_noc/')
    df_qsw_refined.loc[index, 'sent_comp']=sent
    df_qsw_refined.loc[index, 'no_noc']='#'.join(no_noc_list)

In [ ]:
df_qsw_refined.sample(3)

In [ ]:
def vno_mark(vno, sent):
    vno_re=re.compile(vno+'/no_noc/|'+vno+'/ns/')
    vno_list_temp=re.findall(vno_re, sent)
    if vno_list_temp!=[]:
        sent_comp_temp=sent.replace(vno+'/no_noc/', '/vno//no_noc/')
        sent_comp_temp=sent_comp_temp.replace(vno+'/ns/', '/vno//ns/')
        return {'sent_comp':sent_comp_temp, 
                'vno_list':[s.strip('/no_noc/').strip('/ns/') for s in vno_list_temp]}
    else:
        return None

In [ ]:
# Subtract appointing verb (POS:vno).
for index in df_qsw_refined.index:
    sent=df_qsw_refined.loc[index]['sent_comp']
    vno_list=[]
    for vno in df_vno['name']:
        tag_result=vno_mark(vno, sent)
        if tag_result!=None:
            sent=tag_result['sent_comp']
            vno_list+=tag_result['vno_list']
    df_qsw_refined.loc[index, 'sent_comp']=sent
    df_qsw_refined.loc[index, 'vno']='#'.join(vno_list)

In [ ]:
df_qsw_refined.fillna('', inplace=True)
df_qsw_refined.sample(5)

In [ ]:
df_qsw_refined.to_excel('../output/qsw_subtract.xlsx', encoding='utf8')